# Init

In [1]:
import igl
#import meshplot as mp
import polyscope as ps
import vedo as vd
import numpy as np
import time 
import os
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Torsal import Torsal
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

# Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path
math_path = dir_path+"/mathematica/" # mathematica path

In [13]:

def solve_torsal(vi, vj, vk, ei, ej, ek) :

    # Get edges
    vij = vj - vi 
    vik = vk - vi

    eij = ej - ei 
    eik = ek - ei
    

    ec = (ei + ej + ek)/3

    vijxec = np.cross(vij, ec)
    vikxec = np.cross(vik, ec)

    # g0 
    g0 = np.sum(eij*vijxec, axis=1)

    # g1
    g1 = np.sum(eij*vikxec, axis=1) + np.sum(eik*vijxec, axis=1)

    # g2
    g2 = np.sum(eik*vikxec, axis=1)


    disc = g1**2 - 4*g0*g2 

    t1 = np.zeros_like(vij)
    t2 = np.zeros_like(vij)

    a1 = np.zeros(len(vij))
    a2 = np.zeros(len(vij))
    b1 = np.zeros(len(vij))

    # indices disc >0 
    idx = np.where(disc > 0)[0]

    a1[idx] = (-g1[idx] + np.sqrt(g1[idx]**2 - 4*g0[idx]*g2[idx]))
    a2[idx] = (-g1[idx] - np.sqrt(g1[idx]**2 - 4*g0[idx]*g2[idx]))
    b1[idx] = 2*g0[idx]

    # sol
    t1[idx] = (-g1[idx] + np.sqrt(g1[idx]**2 - 4*g0[idx]*g2[idx]))[:, None]*vij[idx] + 2*g0[idx,None]*vik[idx]
    t2[idx] = (-g1[idx] - np.sqrt(g1[idx]**2 - 4*g0[idx]*g2[idx]))[:, None]*vij[idx] + 2*g0[idx,None]*vik[idx]

    # Normalize
    t1[idx] /= np.linalg.norm(t1[idx], axis=1)[:, None]
    t2[idx] /= np.linalg.norm(t2[idx], axis=1)[:, None]

    return t1, t2, a1, a2, b1 

def vv_second(vvi, vvj, vvk, f, numV):

    vv = np.zeros((numV, 3))

    for i in range(len(f)):
        vv[f[i,0]] = vvi[i]
        vv[f[i,1]] = vvj[i]
        vv[f[i,2]] = vvk[i]

    return vv


def init_test_data(data):
    # # Define paths
    dir_path = os.getcwd()
    data_path = dir_path+"/approximation/data/" # data path

    # Data of interest
    k = data

    # Load M mesh (centers of sphere mesh)
    mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 

    # Load test mesh
    tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

    # Create dual mesh
    tmesh = Mesh()
    tmesh.make_mesh(tv,tf)

    # Get inner vertices
    inner_vertices = tmesh.inner_vertices()

    # Get vertex normals for test mesh
    e_i = igl.per_vertex_normals(tv, tf)

    # Fix normal directions
    signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
    e_i = e_i * signs[:, None]

    # Compute circumcenters and axis vectors for each triangle
    p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

    ct, _, nt = circle_3pts(p1, p2, p3)

    # Dual topology 
    dual_tf = tmesh.vertex_ring_faces_list()

    dual_top = tmesh.dual_top()

    # Create hexagonal mesh                            
    h_pts = np.empty((len(tf), 3), dtype=np.float64)
    
    li = np.zeros(len(tf), dtype=np.float64)
    
    center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])

    # Intersect circumcircle axis with center mesh
    for i in range(len(tf)):
        # Get points on circumcircle axis
        p0  = ct[i] - 10*nt[i]
        p1  = ct[i] + 10*nt[i]

        # Get intersection points
        h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

        # Set li 
        li[i] = np.linalg.norm(h_pts[i] - ct[i])

    # Get radius of spheres
    r = np.linalg.norm(h_pts - tv[tf[:,0]], axis=1)

    return tv, tf, ct, nt, li, inner_vertices, e_i, dual_tf, dual_top, r 


def visualize_data(data):
        # # Define paths
    dir_path = os.getcwd()
    data_path = dir_path+"/approximation/data/" # data path

    # Data of interest
    k = data

    # Load M mesh (centers of sphere mesh)
    mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 

    # Load test mesh
    tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

    # Create dual mesh
    tmesh = Mesh()
    tmesh.make_mesh(tv,tf)

    # Get inner vertices
    inner_vertices = tmesh.inner_vertices()

    # Get vertex normals for test mesh
    e_i = igl.per_vertex_normals(tv, tf)

    # Fix normal directions
    signs = np.sign(np.sum(e_i * ([0,0,-1]), axis=1))
    e_i = e_i * signs[:, None]

    # Compute circumcenters and axis vectors for each triangle
    p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

    ct, _, nt = circle_3pts(p1, p2, p3)

    # Dual topology 
    dual_tf = tmesh.vertex_ring_faces_list()

    # Create hexagonal mesh                            
    h_pts = np.empty((len(tf), 3), dtype=np.float64)
    center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])

    ps.init()

    v_mesh = ps.register_surface_mesh("test", tv, tf)

    c_mesh = ps.register_surface_mesh("centers", mv, mf)

    v_mesh.add_vector_quantity("n_t", -nt, defined_on = "faces", enabled=True, length=1.5, color=(0.1, 0.1, 0.1))

    ps.show()




def compute_disc(tv, tf, e_i):

    # # Compute the edge vectors per each face
    vi, vj, vk = tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]]

    # # Compute the edge vectors per each face
    vij = vj - vi
    vik = vk - vi

    # Set up X 
    eij = e_i[tf[:,1]] - e_i[tf[:,0]]
    eik = e_i[tf[:,2]] - e_i[tf[:,0]]

    ec = np.sum( e_i[tf], axis = 1) / 3

    # A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
    # A = gamma11 +  gamma12
    eikXec = np.cross(eik, ec)
    vikXec = np.cross(vik, ec)

    det1 = np.sum(vij*eikXec, axis=1)
    det2 = np.sum(eij*vikXec, axis=1)

    # b = [eij, eik, ec]  c = [vij, vik, ec]

    gamma0 = np.sum(eij*eikXec, axis=1)
    gamma2 = np.sum(vij*vikXec, axis=1)

    A = det1 + det2 

    return A, A**2 - 4*gamma0*gamma2

def unormalize_dir(h_pts, dual, inner_vertices, tv, e_i, rad):
    """Input 
        h_pts: sphere centers
        e_i: edge directions normalized
        rad: sphere radii
        Ouput:
        le: unormalized edge directions

    """
    le = np.ones_like(e_i)
    for i in range(len(inner_vertices)):
        # Get dual faces index
        idx = inner_vertices[i]

        # Get dual face
        f = dual[idx]

        # Get sphere centers
        p = h_pts[f]

        # Get edge direction/'
        e = e_i[idx]

        # Get radius
        r = rad[idx]

        # angle e_i with the direction to the center
        theta = np.arccos(np.sum(e*(p - tv[idx]), axis=1))

        print(theta)

        # Get the lambda
        le[idx] = 2*r*np.cos(theta)

    return le

def planarity_check(t1, tt1, ec):

    t1_tt1 = np.cross(t1, tt1)
    ec /= np.linalg.norm(ec, axis=1)[:, None]
    # Check planarity
    planar = np.sum(t1_tt1*ec, axis=1)

    return planar

# Test

In [24]:


# X: variables [ e   | a | b | n_t  | d_i ] 
# X  size      [ 3*V | F | F | 3*F  | F   ]
# V: Vertices
# F: Faces
# bf: circumcenters of the faces
# ncf: normals of the circumcenters
# w: weight

# Init data 
tv, tf, bt, nt, df, inner_vertices, e_i, dual_tf, dual_top, r = init_test_data(1)

# # Define three vertices
# v = np.array([[-1,-1,-0.2], [1,0,0.1], [-0.8,0.1,0.2]])

# # Define directions ei
# ei = -np.array([[0,0,1], [0,0.1,0.8], [0.1,0,0.9]])

# # Faces
# f = np.array([[0,1,2]])

# Compute the circumcircle
bf, _, ncf = circle_3pts(tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]])

# Init X 
X = np.zeros(3*len(tv) + 11*len(tf))

X[:3*len(tv)] = e_i.flatten()

X[-len(tf):] = df


# Init LineCong
linecong = LineCong()
linecong.initialize_constraint(X, len(tv), bt, nt, len(tf), dual_tf, inner_vertices, 1)

torsal = Torsal()
X = torsal.initialize_constraint(X, tv, tf, bf, ncf, 2)


optimizer = Optimizer()
optimizer.initialize_optimizer(X, "LM", 0.6)

com_t = 0
it = 25
for _ in range(it):
    
    optimizer.add_constraint(torsal, tf)
    optimizer.add_constraint(linecong, dual_tf)
    tic = time.time()
    optimizer.optimize()
    toc = time.time()
    com_t += toc - tic

com_t /= it 

print("Time per iteration: ", com_t)


Mesh Data Structure: |V| = 427, |F| = 774, |E| = 1200
 E 1: 3276.255491638389
 E 2: 687.8935750539516
 E 3: 264.26513500813024
 E 4: 73.95838665872266
 E 5: 24.437159748393533
 E 6: 9.531970277099616
 E 7: 5.10856396828844
 E 8: 2.7494577402895706
 E 9: 1.3823401309989398
 E 10: 0.6063565658872453
 E 11: 0.23896117119319715
 E 12: 0.09752814510752954
 E 13: 0.03589532385578599
 E 14: 0.014699946781568204
 E 15: 0.008096859041618651
 E 16: 0.006155615223814798
 E 17: 0.005593158858506905
 E 18: 0.005422486446473254
 E 19: 0.0053609427436505964
 E 20: 0.005329546936662221
 E 21: 0.005306499415649153
 E 22: 0.005288223943133975
 E 23: 0.005270263773547169
 E 24: 0.005252530343348193
 E 25: 0.005234930187376431
Time per iteration:  6.55760046005249


## Vis

In [31]:

e, a1, b1, nt1, a2, b2, nt2, di = torsal.uncurry_variables(optimizer.X)


vi, vj, vk = tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]]

vik = tv[tf[:,2]] - tv[tf[:,0]]
vij = tv[tf[:,1]] - tv[tf[:,0]]

e = e/np.linalg.norm(e, axis=1)[:, None]

ei, ej, ek = e[tf[:,0]], e[tf[:,1]], e[tf[:,2]]

vvi, vvj, vvk, _, _, _ = torsal.compute_second_env(di, e, tf)

vvij = vvj - vvi
vvik = vvk - vvi

vv = vv_second(vvi, vvj, vvk, tf, len(tv))
# vv = np.array([vvi[0], vvj[0], vvk[0]])

# vvc = np.mean(vv, axis=0)
# vc = np.mean(v, axis=0)

# ec = np.array([vvc - vc])

at1, at2, aa1, aa2, bb = solve_torsal(tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]], ei, ej, ek)

t1 = torsal.compute_t(a1, b1)
t2 = torsal.compute_t(a2, b2)
print("error norms tt1: ", np.linalg.norm( np.linalg.norm(t1, axis=1) - torsal.t1norms))
t1 = t1/torsal.t1norms[:, None]
t2 = t2/torsal.t2norms[:, None]

print("nt1.t1: ", np.sum(nt1*t1, axis=1)@np.sum(nt1*t1, axis=1))

#t1 = opt1/np.linalg.norm(opt1, axis=1)[:, None]
#t2 = opt2/np.linalg.norm(opt2, axis=1)[:, None]

tt1, _, _ = torsal.compute_tt(a1, b1, vvi, vvj, vvk)
tt2, _, _ = torsal.compute_tt(a2, b2, vvi, vvj, vvk)


tt1 = tt1/torsal.tt1norms[:, None]
tt2 = tt2/torsal.tt2norms[:, None]

# tt1 = optt1/np.linalg.norm(optt1, axis=1)[:, None]
# tt2 = optt2/np.linalg.norm(optt2, axis=1)[:, None]

att1 = aa1[:, None]*vvij + bb[:, None]*vvik
att2 = aa2[:, None]*vvij + bb[:, None]*vvik

att1 /= np.linalg.norm(att1, axis=1)[:, None]
att2 /= np.linalg.norm(att2, axis=1)[:, None]

#ec = (ei + ej + ek)/3

vc = (vi + vj + vk)/3
vvc = (vvi + vvj + vvk)/3
ec = vvc - vc

ec2 = torsal.compute_ec(di, e, tf)


# Compute planarity
planar = planarity_check(t1, tt1, ec)

#filter nan values
planar = planar[~np.isnan(planar)]

print(planar@planar)

# Visualization
ps.init()

ps.remove_all_structures()

# Create mesh
triangle = ps.register_surface_mesh("T1", tv, tf)
triangle2 = ps.register_surface_mesh("T2", vv, tf)
#sphere = ps.register_point_cloud("Sphere", bf + di[:,None]*ncf)

# Add directions
#triangle.add_vector_quantity("ei", e, defined_on='vertices', enabled=True, radius=0.005, length=2.0, color=(0.0, 0.0, 0.0))
#triangle.add_vector_quantity("-ei", -e, defined_on='vertices', enabled=True, radius=0.005, length=2.0, color=(0.0, 0.0, 0.0))
#triangle.add_vector_quantity("ec", ec, defined_on='faces', enabled=True, radius=0.0001, length=1.0, color=(0.0, 0.0, 0.0))


# Add torsal directions
triangle.add_vector_quantity("t1",   t1, defined_on="faces", enabled=True, radius=0.001, length=0.008, color=(1.0, 1.0, 1.0))
triangle.add_vector_quantity("-t1", -t1, defined_on="faces", enabled=True, radius=0.001, length=0.008, color=(1.0, 1.0, 1.0))
triangle.add_vector_quantity("t2",   t2, defined_on="faces", enabled=True, radius=0.001, length=0.008, color=(1.0, 1.0, 1.0))
triangle.add_vector_quantity("-t2", -t2, defined_on="faces", enabled=True, radius=0.001, length=0.008, color=(1.0, 1.0, 1.0))



triangle2.add_vector_quantity(" tt1",  tt1, defined_on="faces", enabled=True, radius=0.001, length=0.008, color=(1.0, 1.0, 1.0))
triangle2.add_vector_quantity("-tt1", -tt1, defined_on="faces", enabled=True, radius=0.001, length=0.008, color=(1.0, 1.0, 1.0))
triangle2.add_vector_quantity(" tt2",  tt2, defined_on="faces", enabled=True, radius=0.001, length=0.008, color=(1.0, 1.0, 1.0))
triangle2.add_vector_quantity("-tt2", -tt2, defined_on="faces", enabled=True, radius=0.001, length=0.008, color=(1.0, 1.0, 1.0))
# triangle.add_vector_quantity("at1",  at1, defined_on="faces", enabled=True, radius=0.01, length=0.01, color=(1.0, 0.0, 0.0))
# triangle.add_vector_quantity("at2",  at2, defined_on="faces", enabled=True, radius=0.005, length=0.01, color=(1.0, 0.0, 0.0))
# triangle.add_vector_quantity("-at1", -at1, defined_on="faces", enabled=True, radius=0.01, length=0.01, color=(1.0, 0.0, 0.0))
# triangle.add_vector_quantity("-at2", -at2, defined_on="faces", enabled=True, radius=0.005, length=0.01, color=(1.0, 0.0, 0.0))

# triangle2.add_vector_quantity("tt1",   att1, defined_on="faces", enabled=True, radius=0.01,   length=0.01, color=(0.0, 0.0, 0.0))
# triangle2.add_vector_quantity("tt2",   att2, defined_on="faces", enabled=True, radius=0.01,   length=0.01, color=(0.0, 0.0, 0.0))
# triangle2.add_vector_quantity("-tt1", -att1, defined_on="faces", enabled=True, radius=0.01, length=0.01, color=(0.0, 0.0, 0.0))
# triangle2.add_vector_quantity("-tt2", -att2, defined_on="faces", enabled=True, radius=0.01, length=0.01, color=(0.0, 0.0, 0.0))

#r = np.linalg.norm(bf + di[:,None]*ncf - v[0])
#sphere.set_radius(r, relative=False)

ps.show()


error norms tt1:  0.00046293515432985086
nt1.t1:  0.0005846447587437482
0.0022861469447074396


/tmp/ipykernel_406759/2851724301.py:52: RuntimeWarning: invalid value encountered in divide
  att1 /= np.linalg.norm(att1, axis=1)[:, None]
/tmp/ipykernel_406759/2851724301.py:53: RuntimeWarning: invalid value encountered in divide
  att2 /= np.linalg.norm(att2, axis=1)[:, None]


# LC + Hyp energy Minimization

In [ ]:
visualize_data(1)

In [ ]:
# Init data 
tv, tf, bt, nt, li, inner_vertices, e_i, dual_tf, dual_top, r = init_test_data(1)

# variables = [e_i | A | delta | li]; e_i direction per vertex in T and A is one per each triangle in T, same for delta 
# li is of the lenght of faces
X = np.zeros(3*len(tv) + 3*len(tf) )

# Init directions
X[:3*len(tv)] = e_i.flatten() + np.random.normal(0, 0.1, 3*len(tv))
X[-len(tf): ] = li


# Init LineCong
linecong = LineCong()
linecong.initialize_constraint(X, len(tv), bt, nt, len(tf), dual_tf, inner_vertices, 1)

# # Init Hyperbolic
hyp = HyperbolicLC()
X = hyp.initialize_constraint(X, tv, tf, e_i, 10, 1)

# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X, "LM", 0.8)

for i in range(50):
    # Add constraints to optimizer
    opt.add_constraint(hyp, tf)
    opt.add_constraint(linecong, inner_vertices, dual_tf)

    # Optimize
    opt.optimize()


## Normalize $e_i$

In [ ]:
e_i = opt.X[:3*len(tv)].reshape(len(tv), 3)
# Compute torsal directions 
_, t1, t2, cosang = torsal_dir_vec(tv, tf, e_i)

# Search -1 in cosang
idx = np.where(cosang == -1)[0]

print(idx)

In [ ]:
e_i = opt.X[:3*len(tv)].reshape(len(tv), 3)
e_i /= np.linalg.norm(e_i, axis=1)[:, None]
li = opt.X[-len(tf):]
ct = bt + li[:,None]*nt
#le = unormalize_dir(h_pts, dual_tf, inner_vertices, tv, e_i, r)

In [ ]:
ps.init()

ps.remove_all_structures()

ref      = ps.register_surface_mesh("Reference Mesh", tv, tf, transparency=0.5)
dualmesh = ps.register_surface_mesh("Central mesh"  , ct, dual_top, transparency=0.5)
 
#ref.add_vector_quantity("e_i", -e_i, defined_on = "vertices", enabled=True, length=1.5, color=(0.1, 0.1, 0.1))
ref.add_vector_quantity("axis", nt, defined_on = "faces", enabled=True, length=1.5, color=(0.1, 0.1, 0.1))

ps.show()

## Evaluation

### Export points of evaluation

In [ ]:
# Data of interest
data = 1
 # Export vertices and barycenters
np.savetxt(out_path+"vertices_"+str(data)+".dat", tv)

bar = np.mean(tv[tf], axis=1)
np.savetxt(out_path+"barycenters_"+str(data)+".dat", bar)

### Load Analytic values

In [ ]:
# Analytic torsal directions
at1 = np.loadtxt(math_path+"tdir1.dat")
at2 = np.loadtxt(math_path+"tdir2.dat")
edir = np.loadtxt(math_path+"edir.dat")

# Check dimensions
assert len(edir) == len(e_i), "Not same dimensions in e_i"
assert len(at1) == len(tf), "Not same dimensions in at1"
assert len(at2) == len(tf), "Not same dimensions in at1"

### Compute errors of torsal directions

In [ ]:
def compute_angles_error(t1,t2, at1, at2):
    """
        Function to compute the angle error between two torsal directions.
    """

    # if t1 and t2 are zero vectors, then exclude
    idx = np.where(np.linalg.norm(t1, axis=1) != 0)[0]

    # Compute angle between t1 at1
    ang11 = np.arccos(np.sum(abs(t1*at1), axis=1))

    # Compute angle between t1 at2
    ang12 = np.arccos(np.sum(abs(t1*at2), axis=1))
    
    # Compute angle between t2 at1
    ang21 = np.arccos(np.sum(abs(t2*at1), axis=1))

    # Compute angle between t2 at2
    ang22 = np.arccos(np.sum(abs(t2*at2), axis=1))

    # Compute error as min( 2*(ang11 + ang22)/pi,  2*(ang12 + ang21)/pi)
    error = np.minimum(2*(ang11 + ang22)/np.pi, 2*(ang12 + ang21)/np.pi)


    return error, idx



In [ ]:
# Compute torsal directions 
_, t1, t2, _ = torsal_dir_vec(tv, tf, e_i)

# Compute error as the angle between the analytic and the computed torsal directions
errang, valid_faces = compute_angles_error(t1,t2, at1, at2)

err3 = np.zeros(len(tv))
# Compute error as the angle between the analytic and the computed edge directions
err3[inner_vertices] = np.arccos(np.sum(e_i[inner_vertices]*edir[inner_vertices], axis=1)) * 180/np.pi


In [ ]:
# Show histograms
import matplotlib.pyplot as plt

# Plot two histograms in the same figure but separated
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1 = plt.subplot(1, 2, 1)
ax2 = plt.subplot(1, 2, 2)
# Plot histogram for torsal directions
ax1.hist(errang, bins=20)
ax1.set_xlabel("Angle error")
ax1.set_ylabel("Frequency")
ax1.set_title("Torsal directions")
# Plot histogram for edge directions
ax2.hist(err3, bins=20)
ax2.set_xlabel("Angle error (deg)")
ax2.set_ylabel("Frequency")
ax2.set_title("Edge directions")
plt.show()



### Visualize Torsal directions and errors

In [ ]:
# torsal vis settings


# Init polyscope
ps.init()

# Create mesh
mesh = ps.register_surface_mesh("mesh", tv, tf)

# Add cmap mesh
mesh.add_scalar_quantity("errang", errang, defined_on = "faces", enabled=True, cmap="coolwarm")

# color map vertices with edges error
mesh.add_scalar_quantity("err3", err3, defined_on = "vertices", enabled=True, cmap="coolwarm")

# Add torsal directions
mesh.add_vector_quantity(" at1 ", at1, defined_on ='faces', enabled=True, radius=0.001, length=0.005, color=(0.0, 0.0, 0.0))
mesh.add_vector_quantity("-at1", -at1, defined_on ='faces', enabled=True, radius=0.001, length=0.005, color=(0.0, 0.0, 0.0))
mesh.add_vector_quantity(" at2 ", at2, defined_on ='faces', enabled=True, radius=0.001, length=0.005, color=(0.0, 0.0, 0.0))
mesh.add_vector_quantity("-at2", -at2, defined_on ='faces', enabled=True, radius=0.001, length=0.005, color=(0.0, 0.0, 0.0))

mesh.add_vector_quantity("t1 ",  t1, defined_on ='faces', enabled=True, radius=0.001, length=0.005, color=(1.0, 0.0, 0.0))
mesh.add_vector_quantity("-t1", -t1, defined_on ='faces', enabled=True, radius=0.001, length=0.005, color=(1.0, 0.0, 0.0))
mesh.add_vector_quantity("t2 ",  t2, defined_on ='faces', enabled=True, radius=0.001, length=0.005, color=(1.0, 0.0, 0.0))
mesh.add_vector_quantity("-t2", -t2, defined_on ='faces', enabled=True, radius=0.001, length=0.005, color=(1.0, 0.0, 0.0))

# Visualize directions

# mesh.add_vector_quantity("e_i", e_i, defined_on='vertices', enabled=True, radius=0.001, length=2.0, color=(0.0, 0.0, 0.0))
# # mesh.add_vector_quantity("ae_i", edir, defined_on='vertices', enabled=True, radius=0.001, length=2.0, color=(1.0, 1.0, 1.0))

# # Create second mesh of envelope
# mesh2 = ps.register_surface_mesh("mesh2", tv + 3*e_i, tf)

# mesh3 = ps.register_surface_mesh("mesh3", tv + 3.01*edir, tf)

ps.show()

## Save data

In [ ]:
def export_torsal_direction(data, tv, tf, e_i):

    # Data 
    k = data

    # Create file for positions
    pos = open(os.path.join( out_path, "torsal_pos_"+str(k)+".dat"), "w")
    # Create file for torsal directions
    tdir = open(os.path.join( out_path, "torsal_dir_"+str(k)+".dat"), "w")
    # Create file to store the ids of the inner faces
    f_if = open(os.path.join( out_path, "inner_faces_"+str(k)+".dat"), "w")

    # Compute torsal directions
    barycenters, t1, t2, cos_tors = torsal_dir_vec(tv, tf, e_i)
    
    # Get inner faces
    t_mesh = Mesh()
    t_mesh.make_mesh(tv, tf)
    inner_faces = t_mesh.inner_faces()

    for ii in range(len(inner_faces)):
        i = inner_faces[ii]
        pos.write(str(barycenters[i][0]) + " " + str(barycenters[i][1]) + " " + str(barycenters[i][2]) + "\n")
        tdir.write(str(t1[i][0]) + " " + str(t1[i][1]) + " " + str(t1[i][2]) + " " + str(t2[i][0]) + " " + str(t2[i][1]) + " " + str(t2[i][2]) + "\n")
        f_if.write(str(i) + "\n")


    pos.close()
    print("Positions file created")
    tdir.close()
    print("Torsal dir files created")
    f_if.close()
    print("Inner faces file created")

def export_toral_data_testing(data, tv, tf, e_i, list_bad_faces):

    # Data
    k = data 

    # Compute torsal directions
    _, t1, t2, _ = torsal_dir_vec(tv, tf, e_i)

    # Create data frame
    df = pd.DataFrame()

    # Create a table with e_i[tf[list_bad_faces]], tv[tf[list_bad_Faces]], t1[list_bad_faces], t2[list_bad_faces]
    df["e_i"] = e_i[tf[list_bad_faces]].tolist()
    df["tv"] = tv[tf[list_bad_faces]].tolist()
    df["t1"] = t1[list_bad_faces].tolist()
    df["t2"] = t2[list_bad_faces].tolist()

    # Export df to csv
    df.to_csv(os.path.join( out_path, "bf_data_"+str(k)+".csv"), index=False)




In [ ]:
export_torsal_direction(4, tv, tf, e_i)

In [ ]:
export_toral_data_testing(4, tv, tf, e_i, np.array([847,833,806,776,770,767,733,721,688,571]))

# New Torsal Condition

In [ ]:
# Define three vertices
v = np.array([[-1,-1,-0.2], [1,0,0.1], [-0.8,0.1,0.2]])

# Define directions ei
ei = -np.array([[0,0,1], [0,0.1,0.8], [0.1,0,0.9]])
# Normalize ei
ei /= np.linalg.norm(ei, axis=1)[:, None]

# Compute the circumcircle 
c, cr, n = circle_3pts(v[0], v[1], v[2])

# Define a distance from the circumcenter to the center of a sphere
di = 2

# sphere center
sc = c + di*n

# Compute radius of the sphere
r = np.linalg.norm(v[0] - sc)

# directions vertices to center
dc = sc - v
dc /= np.linalg.norm(dc, axis=1)[:, None] # normalize

dc2 = sc - v

# Second envelope points
angles = np.sum(dc*ei, axis=1)

# ec * dc2 
dotec = np.sum(ei*dc2, axis=1)

# Compute the lambdas
l = 2*r*angles
l2 = 2*dotec

# New points
u  = v + l[:, None]*ei
u2 = v + l2[:, None]*ei

print(f"u:{u}\nu2:{u2}")

# Compute A v = u

A = u.T@np.linalg.inv(v.T)

vij  = v[1] - v[0]
vik  = v[2] - v[0]

uij = u[1] - u[0]
uik = u[2] - u[0]


t1  = 2*vij + 3*vik 
ut1 = 2*uij + 3*uik 
# Print distance u to sc
print(A@t1)
print(ut1)

# Baricenter
vc = np.mean(v, axis=0)
uc = np.mean(u, axis=0)

ec = uc - vc
ec = ec/np.linalg.norm(ec)
ec2 = np.mean(ei, axis=0)
ec2 = ec2/np.linalg.norm(ec2)

print(f"ec : {ec}\nec2: {ec2}")

# Visualization
ps.init()

ps.remove_all_structures()

# Create mesh
triangle = ps.register_surface_mesh("T1", v, np.array([[0,1,2]]))
triangle2 = ps.register_surface_mesh("T2", u, np.array([[0,1,2]]))
sphere = ps.register_point_cloud("Sphere", sc)
sphere.set_radius(r, relative=False)

# Add directions
triangle.add_vector_quantity("ei", ei, defined_on='vertices', enabled=True, radius=0.005, length=2.0, color=(0.0, 0.0, 0.0))
triangle.add_vector_quantity("t1", np.array([t1]), defined_on="faces", enabled=True, radius=0.01, length=0.5, color=(0.0, 0.0, 0.0))
triangle.add_vector_quantity("ec", np.array([ec]), defined_on="faces", enabled=True, radius=0.001, length=2.5, color=(0.8, 0.0, 0.0))
triangle.add_vector_quantity("ec2", np.array([ec2]), defined_on="faces", enabled=True, radius=0.001, length=2.5, color=(0.0, 0.0, 0.8))

triangle2.add_vector_quantity("at1", np.array([ut1]), defined_on="faces", enabled=True, radius=0.01, length=0.5, color=(0.0, 0.0, 0.0))

ps.show()




## Visualization